In [1]:
import cv2
import argparse

from ultralytics import YOLO
import supervision as sv
import numpy as np

In [2]:
ZONE_POLYGON = np.array([
    [0, 0],
    [1, 0],
    [1, 1],
    [0, 1]
])

In [3]:
#parse_arguments()

In [4]:
video_path = "sandwich_test3.mp4"
webcam_resolution = [1920, 1080]

In [5]:
labels1 = []
detections1 = []
masses = []

In [6]:
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

In [7]:
def main():
    #args = parse_arguments()
    frame_width, frame_height = webcam_resolution

    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

    model = YOLO("best.pt")

    box_annotator = sv.BoxAnnotator(
        thickness=2,
        text_thickness=2,
        text_scale=1
    )

    zone_polygon = (ZONE_POLYGON * np.array(webcam_resolution)).astype(int)
    zone = sv.PolygonZone(polygon=zone_polygon, frame_resolution_wh=tuple(webcam_resolution))
    zone_annotator = sv.PolygonZoneAnnotator(
        zone=zone, 
        color=sv.Color.red(),
        thickness=2,
        text_thickness=4,
        text_scale=2
    )

    while cap.isOpened():
        ret, frame = cap.read()

        result = model(frame, agnostic_nms=True)[0]
        detections = sv.Detections.from_yolov8(result)
        detections1.append(detections)
        labels = [
            f"{model.model.names[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, _
            in detections
        ]
        labels1.append(labels)
        #Surface Area
        label_sa = []
        try:
            masses_li = []
            for i in range(0, len(labels)):
                mylabel = labels[i]
                xyxy = detections.xyxy[i]
                width = (xyxy[2] - xyxy[0])/100
                height = (xyxy[3] - xyxy[1])/100
                area = width * height
                if mylabel == 'Bread':
                    normal_thickness = 1.8 #Average
                    density = 0.25
                    mass = area*normal_thickness*density
                elif mylabel == 'Bacon':
                    normal_thickness = 1.5
                    density = 0.8
                    mass = area*normal_thickness*density
                elif mylabel == 'Tomato':
                    normal_thickness = 0.85
                    density = 0.9
                    mass = area*normal_thickness*density
                else:
                    normal_thickness = 0.32
                    density = 0.6
                    mass = area*normal_thickness*density                    
                
                
                mylabel_sa = labels[i] + ' , Mass: {}'.format(round(mass, 4))
                label_sa.append(mylabel_sa)
                masses_li.append(mass)
            mean_mass = np.mean(masses_li)
            masses.append(mean_mass)
                
            frame = box_annotator.annotate(
                scene=frame, 
                detections=detections, 
                labels=label_sa
            )
        except IndexError:
            pass
        zone.trigger(detections=detections)
        frame = zone_annotator.annotate(scene=frame) 
        
        # Write the frame
        out.write(frame)
        # Show the frame
        cv2.imshow("yolov8", frame)

        if (cv2.waitKey(30) == 27):
            break
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

Ultralytics YOLOv8.0.32 🚀 Python-3.10.13 torch-2.0.1+cu117 CPU
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs

0: 480x800 184.1ms
Speed: 2.4ms pre-process, 184.1ms inference, 0.5ms postprocess per image at shape (1, 3, 800, 800)
/home/festo/anaconda3/envs/cv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/festo/anaconda3/envs/cv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)

0: 480x800 258.7ms
Speed: 1.3ms pre-process, 258.7ms inference, 0.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 188.5ms
Speed: 1.4ms pre-process, 188.5ms inference, 0.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 179.7ms
Speed: 0.9ms pre-process, 179.7ms inference, 0.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 179.7m


0: 480x800 1 Bread, 165.9ms
Speed: 0.8ms pre-process, 165.9ms inference, 0.7ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bacon, 1 Bread, 181.4ms
Speed: 0.9ms pre-process, 181.4ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 174.0ms
Speed: 0.8ms pre-process, 174.0ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bacon, 1 Bread, 174.7ms
Speed: 0.9ms pre-process, 174.7ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bacon, 1 Bread, 177.6ms
Speed: 0.8ms pre-process, 177.6ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 207.0ms
Speed: 1.0ms pre-process, 207.0ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 170.3ms
Speed: 0.9ms pre-process, 170.3ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 193.1ms
Speed: 1.4ms pre-process, 193.1ms inference, 0.8


0: 480x800 1 Bread, 302.5ms
Speed: 0.8ms pre-process, 302.5ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 181.2ms
Speed: 0.8ms pre-process, 181.2ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 303.8ms
Speed: 1.5ms pre-process, 303.8ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 284.5ms
Speed: 0.9ms pre-process, 284.5ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 218.2ms
Speed: 1.3ms pre-process, 218.2ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 283.2ms
Speed: 2.1ms pre-process, 283.2ms inference, 1.1ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 275.3ms
Speed: 0.9ms pre-process, 275.3ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 284.5ms
Speed: 0.9ms pre-process, 284.5ms inference, 1.5ms postprocess per image at

0: 480x800 1 Bread, 341.8ms
Speed: 0.9ms pre-process, 341.8ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 416.3ms
Speed: 1.0ms pre-process, 416.3ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 280.5ms
Speed: 0.9ms pre-process, 280.5ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 335.9ms
Speed: 0.9ms pre-process, 335.9ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 412.0ms
Speed: 0.9ms pre-process, 412.0ms inference, 9.7ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 360.8ms
Speed: 1.0ms pre-process, 360.8ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 480.9ms
Speed: 0.9ms pre-process, 480.9ms inference, 6.6ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 335.4ms
Speed: 2.8ms pre-process, 335.4ms inference, 2.4ms postprocess per image at 


0: 480x800 293.3ms
Speed: 1.2ms pre-process, 293.3ms inference, 0.7ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 281.9ms
Speed: 0.8ms pre-process, 281.9ms inference, 0.7ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 2 Bacons, 322.5ms
Speed: 1.0ms pre-process, 322.5ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bacon, 301.1ms
Speed: 1.0ms pre-process, 301.1ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bacon, 287.9ms
Speed: 0.9ms pre-process, 287.9ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 2 Bacons, 308.1ms
Speed: 0.8ms pre-process, 308.1ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 2 Bacons, 313.5ms
Speed: 0.9ms pre-process, 313.5ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 4 Bacons, 278.9ms
Speed: 0.9ms pre-process, 278.9ms inference, 1.5ms postprocess per image at shape (1, 3, 

...

Speed: 0.9ms pre-process, 334.5ms inference, 2.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Lettuce, 315.1ms
Speed: 0.9ms pre-process, 315.1ms inference, 1.3ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Lettuce, 283.8ms
Speed: 0.9ms pre-process, 283.8ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Lettuce, 334.8ms
Speed: 1.0ms pre-process, 334.8ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Lettuce, 325.6ms
Speed: 1.0ms pre-process, 325.6ms inference, 1.3ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Lettuce, 314.5ms
Speed: 0.9ms pre-process, 314.5ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Lettuce, 408.1ms
Speed: 1.1ms pre-process, 408.1ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Lettuce, 260.8ms
Speed: 0.9ms pre-process, 260.8ms inference, 1.5ms postprocess per image at shape (1, 3, 8


0: 480x800 1 Bacon, 345.3ms
Speed: 1.1ms pre-process, 345.3ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 2 Bacons, 424.8ms
Speed: 1.1ms pre-process, 424.8ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 5 Bacons, 277.7ms
Speed: 0.9ms pre-process, 277.7ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 4 Bacons, 293.3ms
Speed: 0.9ms pre-process, 293.3ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 3 Bacons, 358.3ms
Speed: 0.9ms pre-process, 358.3ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 4 Bacons, 310.0ms
Speed: 1.0ms pre-process, 310.0ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 5 Bacons, 299.5ms
Speed: 1.0ms pre-process, 299.5ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 5 Bacons, 320.9ms
Speed: 0.9ms pre-process, 320.9ms inference, 1.4ms postprocess per i


0: 480x800 1 Bread, 306.8ms
Speed: 1.2ms pre-process, 306.8ms inference, 1.1ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 304.1ms
Speed: 0.9ms pre-process, 304.1ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 281.8ms
Speed: 0.9ms pre-process, 281.8ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 252.5ms
Speed: 0.8ms pre-process, 252.5ms inference, 1.3ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 304.3ms
Speed: 0.9ms pre-process, 304.3ms inference, 1.2ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 314.3ms
Speed: 0.9ms pre-process, 314.3ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 277.5ms
Speed: 1.0ms pre-process, 277.5ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 292.5ms
Speed: 0.9ms pre-process, 292.5ms inference, 1.4ms postprocess per image at


0: 480x800 1 Bread, 344.3ms
Speed: 0.9ms pre-process, 344.3ms inference, 1.7ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 412.5ms
Speed: 1.0ms pre-process, 412.5ms inference, 1.8ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 444.4ms
Speed: 0.9ms pre-process, 444.4ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 434.9ms
Speed: 0.9ms pre-process, 434.9ms inference, 1.7ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 314.7ms
Speed: 0.9ms pre-process, 314.7ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 340.8ms
Speed: 0.8ms pre-process, 340.8ms inference, 1.6ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 424.2ms
Speed: 1.0ms pre-process, 424.2ms inference, 1.4ms postprocess per image at shape (1, 3, 800, 800)

0: 480x800 1 Bread, 446.2ms
Speed: 2.8ms pre-process, 446.2ms inference, 1.6ms postprocess per image at

FileNotFoundError: /media/festo/backup/Essex/Praveen/yolov8-live-master/ultralytics/assets does not exist

In [8]:
labels1

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Bread 0.57'],
 ['Bread 0.76'],
 ['Bread 0.75'],
 ['Bread 0.78'],
 ['Bread 0.77'],
 ['Bread 0.84'],
 ['Bread 0.83'],
 ['Bread 0.87', 'Bacon 0.43', 'Bacon 0.31'],
 ['Bread 0.87', 'Bacon 0.43', 'Bacon 0.31'],
 ['Bread 0.87', 'Bacon 0.45', 'Bacon 0.36'],
 ['Bread 0.87', 'Bacon 0.43', 'Bacon 0.35'],
 ['Bread 0.87', 'Bacon 0.49', 'Bacon 0.39'],
 ['Bread 0.88', 'Bacon 0.38'],
 ['Bread 0.87', 'Bacon 0.41', 'Bacon 0.31'],
 ['Bread 0.85', 'Bacon 0.44'],
 ['Bread 0.85', 'Bacon 0.31'],
 ['Bread 0.83', 'Bacon 0.35'],
 ['Bread 0.84', 'Bacon 0.34'],
 ['Bread 0.84', 'Bacon 0.36'],
 ['Bread 0.83', 'Bacon 0.30'],
 ['Bread 0.83'],
 ['Bread 0.84', 'Bacon 0.27'],
 ['Bread 0.84'],
 ['Bread 0.86', 'Bacon 0.26'],
 ['Bread 0.85'],
 ['Bread 0.86', 'Bacon 0.37'],
 ['Bread 0.85', 'Bacon 0.33'],
 ['Bread 0.85'],
 ['Brea

In [9]:
labels1[-50]

['Bread 0.90']

In [10]:
detections1[-50]

Detections(xyxy=array([[        495,           0,        1617,        1077]], dtype=float32), confidence=array([    0.90125], dtype=float32), class_id=array([1]), tracker_id=None)

In [11]:
xyxy = detections1[-50].xyxy[0]
xyxy

array([        495,           0,        1617,        1077], dtype=float32)

In [12]:
width = (xyxy[2] - xyxy[0])/100
height = (xyxy[3] - xyxy[1])/100
normal_thickness = 1.8
area = width * height
mass = area*normal_thickness*0.25

print("The Area of the object(cm) is:", area)
print("The Mass of the object(cm) is:", mass)

The Area of the object(cm) is: 120.8394
The Mass of the object(cm) is: 54.37773


In [13]:
masses[-50]

23.201164799999997

In [14]:
width

11.22

In [15]:
height

10.77

## Finding Number of Ingredients

In [16]:
from statistics import mode
def findnum_ingredients(ingredient):
    # Filter lists to include only ingredient_i labels
    ingredient_lists = [lst for lst in labels1 if all(ingredient in item for item in lst)]
    my_ingredient = [list_of_list for list_of_list in ingredient_lists if list_of_list] #SRemove any empty list if any

    # Calculate the length of each list
    lengths = [len(lst) for lst in my_ingredient]

    # Find the mode length
    mode_length = mode(lengths)
    if ingredient == 'Bread':
        mode_length = mode_length*2 #Top and bottom bread        
    return mode_length

In [17]:
num_bread = findnum_ingredients('Bread')
num_bacon = findnum_ingredients('Bacon')
num_tomato = findnum_ingredients('Tomato')
num_lettuce = findnum_ingredients('Lettuce')

In [18]:
print(f'Bread: {num_bread}')
print(f'Bacon: {num_bacon}')
print(f'Tomato: {num_tomato}')
print(f'Lettuce: {num_lettuce}')

Bread: 2
Bacon: 4
Tomato: 4
Lettuce: 1


## Average Mass of Ingredients

In [19]:
#We create a function to find indices of those lists whose labels are only the required ingredient
def find_ingredient_list_indices(lists, ingredient):
    bacon_list_indices = []
    for i, list_ in enumerate(lists):
        if all(item.startswith(ingredient) for item in list_):
            bacon_list_indices.append(i)
    return bacon_list_indices

In [20]:
bacon_indices = find_ingredient_list_indices(labels1, 'Bacon')

In [21]:
print(bacon_indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 212, 213, 214, 219, 232, 233, 234, 235, 236, 237, 242, 248, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 283, 284, 285, 286, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 304, 305, 309, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 356, 358, 359, 360, 368, 369, 370, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448

In [22]:
labels1[bacon_indices[100]]

['Bacon 0.62']

In [43]:
def ingredient_mass(my_ingredient):
    ingredient_indices = find_ingredient_list_indices(labels1, my_ingredient) #Get indices for required lists
    ingredient_mass_list = []
    for index in ingredient_indices:
        corresponding_mass = masses[index]
        ingredient_mass_list.append(corresponding_mass) #Save ingredient mass to list
    ingredient_mass_list = [x for x in ingredient_mass_list if np.isnan(x) == False] #Remove nan values
    my_mass = np.mean(ingredient_mass_list)
    if my_ingredient == 'Bread':
        my_mass = np.mean(ingredient_mass_list)*2
    return my_mass

In [44]:
test = ingredient_mass('Bread')
test

43.56771423396227

In [45]:
bread_mass = ingredient_mass('Bread')
bacon_mass = ingredient_mass('Bacon')
tomato_mass = ingredient_mass('Tomato')
lettuce_mass = ingredient_mass('Lettuce')
total_mass = bread_mass + bacon_mass + tomato_mass + lettuce_mass

In [46]:
print('Bread mass: {}'.format(bread_mass))
print('Bacon mass: {}'.format(bacon_mass))
print('Tomato mass: {}'.format(tomato_mass))
print('Lettuce mass: {}'.format(lettuce_mass))

Bread mass: 43.56771423396227
Bacon mass: 11.760488158596491
Tomato mass: 7.981275105664739
Lettuce mass: 20.72495465915493


## Ingredient Nutritional Value

In [47]:
num_bread = findnum_ingredients('Bread')
num_bacon = findnum_ingredients('Bacon')
num_tomato = findnum_ingredients('Tomato')
num_lettuce = findnum_ingredients('Lettuce')

In [48]:
bread_mass = ingredient_mass('Bread')
bacon_mass = ingredient_mass('Bacon')
tomato_mass = ingredient_mass('Tomato')
lettuce_mass = ingredient_mass('Lettuce')

In [49]:
#The data provided gives the nutritional value per 100g, t
#Bread Nutritional Value per 1g
bread_energy_kj_100g = 931/100
bread_energy_kcal_100g = 219/100
bread_fat_g_100g = 1.6/100
bread_saturates_g_100g = 0.29/100
bread_sugars_100g = 3.4/100
bread_protein_100g = 7.9/100
bread_salt_100g = 1.13/100

#Bacon Nutritional Value per 1g
bacon_energy_kj_100g = 1400/100
bacon_energy_kcal_100g = 337/100
bacon_fat_g_100g = 26.9/100
bacon_saturates_g_100g = 9.8/100
bacon_sugars_100g = 0
bacon_protein_100g = 23.8
bacon_salt_100g = 4.14

#Tomato Nutritional Value per 1g
tomato_energy_kj_100g = 61
tomato_energy_kcal_100g = 14
tomato_fat_g_100g = 0.1
tomato_saturates_g_100g = 0.03
tomato_sugars_100g = 3
tomato_protein_100g = 0.5
tomato_salt_100g = 0.13

#Lettuce Nutritional Value per 1g
lettuce_energy_kj_100g = 48
lettuce_energy_kcal_100g = 11
lettuce_fat_g_100g = 0.1
lettuce_saturates_g_100g = 0.03
lettuce_sugars_100g = 1.4
lettuce_protein_100g = 1.2
lettuce_salt_100g = 0.12

In [50]:
# Bread Total Nutritional Value
# We divide all values by 100 since the nutitional values are provided per 100g
bread_energy_kj = num_bread * bread_mass * bread_energy_kj_100g
bread_energy_kcal = num_bread * bread_mass * bread_energy_kcal_100g
bread_fat_g = num_bread * bread_mass * bread_fat_g_100g
bread_saturates_g = num_bread * bread_mass * bread_saturates_g_100g
bread_sugars = num_bread * bread_mass * bread_sugars_100g
bread_protein = num_bread * bread_mass * bread_protein_100g
bread_salt = num_bread * bread_mass * bread_salt_100g

#Bacon Total Nutritional Value 
bacon_energy_kj = num_bacon * bacon_mass * bacon_energy_kj_100g
bacon_energy_kcal = num_bacon * bacon_mass * bacon_energy_kcal_100g
bacon_fat_g = num_bacon * bacon_mass * bacon_fat_g_100g
bacon_saturates_g = num_bacon * bacon_mass * bacon_saturates_g_100g
bacon_sugars = num_bacon * bacon_mass * bacon_sugars_100g
bacon_protein = num_bacon * bacon_mass * bacon_protein_100g
bacon_salt = num_bacon * bacon_mass * bacon_salt_100g

#Tomato Total Nutritional Value 
tomato_energy_kj = num_tomato * tomato_mass * tomato_energy_kj_100g
tomato_energy_kcal = num_tomato * tomato_mass * tomato_energy_kcal_100g
tomato_fat_g = num_tomato * tomato_mass * tomato_fat_g_100g
tomato_saturates_g = num_tomato * tomato_mass * tomato_saturates_g_100g
tomato_sugars = num_tomato * tomato_mass * tomato_sugars_100g
tomato_protein = num_tomato * tomato_mass * tomato_protein_100g
tomato_salt = num_tomato * tomato_mass * tomato_salt_100g

#Lettuce Total Nutritional Value 
lettuce_energy_kj = num_lettuce * lettuce_mass * lettuce_energy_kj_100g
lettuce_energy_kcal = num_lettuce * lettuce_mass * lettuce_energy_kcal_100g
lettuce_fat_g = num_lettuce * lettuce_mass * lettuce_fat_g_100g
lettuce_saturates_g = num_lettuce * lettuce_mass * lettuce_saturates_g_100g
lettuce_sugars= num_lettuce * lettuce_mass * lettuce_sugars_100g
lettuce_protein = num_lettuce * lettuce_mass * lettuce_protein_100g
lettuce_salt_ = num_lettuce * lettuce_mass * lettuce_salt_100g

In [51]:
# Nutritional Values of the whole Sandwich
energy_kj = bread_energy_kj + bacon_energy_kj + tomato_energy_kj + lettuce_energy_kj
energy_kcal = bread_energy_kcal + bacon_energy_kcal + tomato_energy_kcal + lettuce_energy_kcal
fat_g = bread_fat_g + bacon_fat_g + tomato_fat_g + lettuce_fat_g
saturates_g = bread_saturates_g + bacon_saturates_g + tomato_saturates_g + lettuce_saturates_g
sugars = bread_sugars + bacon_sugars + tomato_sugars + lettuce_sugars
protein = bread_protein + bacon_protein + tomato_protein + lettuce_protein
salt = bread_salt + bacon_salt + tomato_salt + lettuce_salt_

In [52]:
print('Energy (kj)    : {}'.format(energy_kj))
print('Energy (kcal)  : {}'.format(energy_kcal))
print('Fat (g)        : {}'.format(fat_g))
print('Saturates (g)  : {}'.format(saturates_g))
print('Sugars  (g)    : {}'.format(sugars))
print('Protein (g)    : {}'.format(protein))
print('Salt (g)       : {}'.format(salt))

Energy (kj)    : 4412.0471253394135
Energy (kcal)  : 1024.283875890565
Fat (g)        : 19.313457622318005
Saturates (g)  : 6.442305753181222
Sugars  (g)    : 127.7528423587032
Protein (g)    : 1167.3146673496674
Salt (g)       : 202.37557186208969


In [53]:
cost_bread = bread_mass*0.0022 #0.0022 is the cost per gram
cost_bacon = bacon_mass*0.00221 #0.00221 is the cost per gram
cost_tomato = tomato_mass*0.00566 #0.00566 is the cost per gram
cost_lettuce = lettuce_mass*0.00482 #0.00482 is the cost per gram
total_cost = cost_bread + cost_bacon + cost_tomato + cost_lettuce

In [58]:
bread_rating = bread_mass/45*10
bacon_rating = bacon_mass/12*10
tomato_rating = tomato_mass/8.5*10
lettuce_rating = lettuce_mass/22*10
sandwich_rating = (bread_rating + bacon_rating + tomato_rating + lettuce_rating)/4

In [59]:
tomato_mass/8

0.9976593882080924

In [60]:
import pandas as pd
#Ingredients
pd.DataFrame({'Item': ['Detected', 'Count', 'Weight (g)'],
              'Bread': ['Yes', round(num_bread, 4), round(bread_mass, 4)],
              'Bacon': ['Yes', round(num_bacon, 4), round(bacon_mass, 4)],
              'Tomato':['Yes', round(num_tomato, 4), round(tomato_mass, 4)],
              'Lettuce': ['Yes', round(num_lettuce, 4), round(lettuce_mass, 4)]             
             })

,Item,Bread,Bacon,Tomato,Lettuce
0,Detected,Yes,Yes,Yes,Yes
1,Count,2,4,4,1
2,Weight (g),43.5677,11.7605,7.9813,20.725


In [61]:
pd.DataFrame({'Item': ['Rating','Energy (kj)', 'Energy (kcal)', 'Fat (g)', 'Saturates (g)', 'Sugars (g)',
                       'Protein (g)', 'Salt (g)', 'Cost'],
              'Bread': [bread_rating, bread_energy_kj, bread_energy_kcal, bread_fat_g, bread_saturates_g, 
                        bread_sugars, bread_protein, bread_salt, cost_bread],
              'Bacon': [bacon_rating, bacon_energy_kj, bacon_energy_kcal, bacon_fat_g, bacon_saturates_g, 
                        bacon_sugars, bacon_protein, bacon_salt, cost_bacon],
              'Tomato':[tomato_rating, tomato_energy_kj, tomato_energy_kcal, tomato_fat_g, tomato_saturates_g, 
                       tomato_sugars, tomato_protein ,tomato_salt, cost_tomato],
              'Lettuce': [lettuce_rating, lettuce_energy_kj, lettuce_energy_kcal, lettuce_fat_g, lettuce_saturates_g, 
                          lettuce_sugars, lettuce_protein, lettuce_salt_, cost_lettuce], 
              'Sandwich': [sandwich_rating, energy_kj, energy_kcal, fat_g, saturates_g, sugars, protein, salt, total_cost]
             })

,Item,Bread,Bacon,Tomato,Lettuce,Sandwich
0,Rating,9.681714,9.800407,9.389735,9.420434,9.573073
1,Energy (kj),811.230839,658.587337,1947.431126,994.797824,4412.047125
2,Energy (kcal),190.826588,158.531380,446.951406,227.974501,1024.283876
3,Fat (g),1.394167,12.654285,3.192510,2.072495,19.313458
4,Saturates (g),0.252693,4.610111,0.957753,0.621749,6.442306
5,Sugars (g),2.962605,0.000000,95.775301,29.014937,127.752842
6,Protein (g),6.883699,1119.598473,15.962550,24.869946,1167.314667
7,Salt (g),0.984630,194.753684,4.150263,2.486995,202.375572
8,Cost,0.095849,0.025991,0.045174,0.099894,0.266908
